# Connecting to VM

In [16]:
from chi import server, storage, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [5]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-persist-mlops-project3", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium",
)
s.submit(idempotent=True)

Waiting for server node-persist-mlops-project3's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


In [6]:
s.associate_floating_ip()

In [7]:
s.refresh()
s.show(type="widget")

In [8]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by MLFlow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"}
]

In [9]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

updated security groups: ['allow-8000', 'allow-8888', 'allow-9000', 'allow-9001', 'allow-ssh', 'default']


In [13]:
s.refresh()
s.check_connectivity()

Checking connectivity to None port 22.


ResourceError: Waited too long for the port 22 on host None to start accepting connections.

# Create Persistent Storage

In [19]:
# get the Cinder Python client configured by python-chi
cinder_client = chi.clients.cinder()

In [20]:
# list current volumes
cinder_client.volumes.list()

[<Volume: 7c5d0219-3559-4910-9fe2-d2e9f702c13b>,
 <Volume: 6bb3854b-244d-46c1-bd0d-3e22f9e364ef>,
 <Volume: 10798efb-e3f4-4410-a1f8-ee6d9ece5de9>,
 <Volume: e9d59d73-3c67-4aa6-ae52-07b2f6c5fd83>,
 <Volume: 6ecc51a4-93bf-4f11-8c96-84f86e1a1535>,
 <Volume: cf7bb2f1-e28f-4491-96f4-3e95f551d306>,
 <Volume: 0b4b905a-55c8-4327-8a7a-ccd29c0b0ac6>,
 <Volume: e087ee06-c76a-49b2-85f7-0d510e28f719>,
 <Volume: 6aaaca52-bbb6-4e7d-bdeb-0dd6da124928>,
 <Volume: d6add162-da8f-42c2-a9ff-42af5e245dad>,
 <Volume: 91e29152-ea3c-4663-a051-db6d04c91eca>,
 <Volume: 492d9a47-1c3b-48be-b78d-9620a3cb863c>,
 <Volume: a5932e82-cc78-4fb3-a01b-9bf280a0afc2>,
 <Volume: c2757059-dab7-4e7d-98bb-1a840a6ea2a6>,
 <Volume: 05bc8fd5-1078-4aaa-8a57-93449c2c14c5>,
 <Volume: 45778fe9-4530-43de-a16f-72a381c4776e>,
 <Volume: b80409e3-dcb0-499c-b66a-7348c061db69>,
 <Volume: fe4d66a5-86a7-4c10-b6e9-167cab29df77>,
 <Volume: a36de9d5-a9dc-443e-bb92-5f762e10d271>,
 <Volume: 3f015e6f-f6f7-47d3-bfdf-3fbe680870f0>,
 <Volume: 779b2fbb-a

In [21]:
# create a volume, specifying name and size in GiB
volume = cinder_client.volumes.create(name=f"block-persist-python-project2", size = 20)
volume._info

{'id': 'd2e7a58c-a3c1-42b6-8498-f12ef14e2b0c',
 'status': 'creating',
 'size': 20,
 'availability_zone': 'nova',
 'created_at': '2025-05-06T18:49:19.275755',
 'updated_at': None,
 'name': 'block-persist-python-project2',
 'description': None,
 'volume_type': 'ceph-hdd',
 'snapshot_id': None,
 'source_volid': None,
 'metadata': {},
 'links': [{'rel': 'self',
   'href': 'https://kvm.tacc.chameleoncloud.org:8776/v3/89f528973fea4b3a981f9b2344e522de/volumes/d2e7a58c-a3c1-42b6-8498-f12ef14e2b0c'},
  {'rel': 'bookmark',
   'href': 'https://kvm.tacc.chameleoncloud.org:8776/89f528973fea4b3a981f9b2344e522de/volumes/d2e7a58c-a3c1-42b6-8498-f12ef14e2b0c'}],
 'user_id': '3772a2b3c4c9ef99560c8940f7b8e754f6388a2d5f9c4f42691ed173013f8d3f',
 'bootable': 'false',
 'encrypted': False,
 'replication_status': None,
 'consistencygroup_id': None,
 'multiattach': False,
 'attachments': []}

In [26]:
server_id      = chi.server.get_server(f"node-persist-mlops-project3").id

In [27]:
volume_manager = chi.nova().volumes
volume_manager.create_server_volume(server_id = s.id, volume_id = volume.id)

<Volume: d2e7a58c-a3c1-42b6-8498-f12ef14e2b0c>

In [ ]:
#volume_manager.delete_server_volume(server_id = s.id, volume_id = volume.id)